In [1]:
import requests
import re
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
import time
import pandas as pd
from langchain_chroma import Chroma
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json
import csv

In [2]:
# ONOS Controller Details
ONOS_BASE_URL = "http://10.23.7.63:8181/onos/v1/flows"
USERNAME = "onos"
PASSWORD = "rocks"  # Replace with your ONOS credentials

# Define sudo password
sudo_password = "test@irciss008"

In [3]:
def get_flows_for_device_ONOS(device_id):
    """
    Retrieve all flows for a specific ONOS device (switch).
    
    Parameters:
        device_id (str): The ONOS switch ID (e.g., "of:0000000000000002").
    
    Returns:
        list: A list of flow JSON objects for the given switch.
    """
    url = f"{ONOS_BASE_URL}/{device_id}"
    headers = {"Accept": "application/json"}

    try:
        response = requests.get(url, auth=(USERNAME, PASSWORD), headers=headers)
        response.raise_for_status()
        
        data = response.json()
        return data.get("flows", [])

    except requests.exceptions.RequestException as e:
        print(f"Error fetching flows for device {device_id}:", e)
        return []

def extract_switch_id_ONOS(intent: str):
    """
    Extract the switch ID from a natural language intent for ONOS JSON format.
    
    Parameters:
        intent (str): The natural language intent.
    
    Returns:
        str: Extracted switch ID (e.g., 'of:0000000000000001') or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match patterns like 'switch 1', 'router 2', 'node 3'
    match = re.search(r'\b(?:switch|router|node|device)(?:\s*number)?\s*(\d+)', intent, re.IGNORECASE)
    if match:
        switch_number = int(match.group(1))
        return f"of:{switch_number:016x}"  # Convert to ONOS 16-digit hex format

    # Match ordinal words (e.g., 'fourth switch', 'second router')
    match = re.search(r'\b(?:switch|router|node|device)\s*(\w+)', intent, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            switch_number = ordinals[ordinal_word]
            return f"of:{switch_number:016x}"  # Convert to ONOS 16-digit hex format

    # Match standalone ordinal words (e.g., 'fourth' without 'switch')
    for word, number in ordinals.items():
        if word in intent.lower():
            return f"of:{number:016x}"  # Convert to ONOS 16-digit hex format

    return None

def push_flow_rule(device_id, flow_json):
    """
    Push a flow rule to ONOS and retrieve the flow ID.
    """
    url = f"{ONOS_BASE_URL}"
    HEADERS = { 
        "Content-Type": "application/json",
        "Accept": "application/json"
        }
    
    try:
        response = requests.post(url, headers=HEADERS, auth=(USERNAME, PASSWORD), data=json.dumps(flow_json))
        if response.status_code in [200, 201]:
            response_data = response.json()
            if "flows" in response_data and len(response_data["flows"]) > 0:
                flow_id = response_data["flows"][0].get("flowId")
                print(f"Successfully pushed flow rule to ONOS (Device ID: {device_id}, Flow ID: {flow_id})")
                return flow_id  # Return the flow ID for later verification
            else:
                print("Flow rule pushed but no Flow ID returned by ONOS.")
                return None
        else:
            print(f"Failed to push flow rule to ONOS. Status Code: {response.status_code}, Response: {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred while pushing flow rule: {e}")
        return None
    
def verify_flow_rule(device_id, flow_id):
    """
    Verify if the flow rule exists in ONOS using the retrieved Flow ID.
    """
    if not flow_id:
        print("\nInvalid Flow ID provided for verification. Check ONOS response after rule posting.\n")
        return False

    url = f"{ONOS_BASE_URL}/{device_id}"
    HEADERS = {"Accept": "application/json"}
    
    try:
        response = requests.get(url, headers=HEADERS, auth=(USERNAME, PASSWORD))
        if response.status_code == 200:
            flows = response.json().get("flows", [])
            for flow in flows:
                if flow.get("id") == flow_id:
                    print(f"Flow rule exists in ONOS (Flow ID: {flow_id})")
                    return True
            print(f"Flow rule NOT found in ONOS (Flow ID: {flow_id})")
            return False
        else:
            print(f"Failed to query ONOS. Status Code: {response.status_code}, Response: {response.text}")
            return False
    except Exception as e:
        print(f"Exception occurred while verifying flow rule: {e}")
        return False

In [ ]:
TRANSLATION_PROMPT_ONOS = """Your task is to transform natural language network intents into JSON-formatted network policies compatible with the ONOS SDN controller.

You only reply in JSON, no natural language. The network intents can represent different traffic control behaviors, such as:

1. **Traffic Forwarding, Queue Assignment, and VLAN Rules:** Define rules for forwarding traffic based on IPv4/IPv6 destination, TCP/UDP ports, and optionally assign traffic to specific queues or vlans.
2. **Blocking or Dropping Rule:** Define rules to drop traffic based on specific match criteria (e.g., source IP, destination IP). In ONOS, this is done by omitting the `"treatment"` field.

### **JSON STRUCTUREs FOR ONOS**

1. **Traffic Forwarding, Queue Assignment, and VLAN Rules:**  

```json
{
    "flows": [
        {
            "priority": <integer>,
            "timeout": <integer>, // Default: 0
            "isPermanent": "true",
            "deviceId": "<switch_id>",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": <integer>
                    },
                    {
                        "type": "L2MODIFICATION",
                        "subtype": "VLAN_ID",
                        "vlanId": <integer> // Example: 100 for VLAN tagging
                    },
                    {
                        "type": "OUTPUT",
                        "port": "<integer>"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "<string>" // Example: "0x800" for IPv4
                    },
                    {
                        "type": "IPV4_SRC",
                        "ip": "<ip_address/mask>"
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "<ip_address/mask>"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": <integer> // Example: 6 for TCP, 17 for UDP
                    },
                    {
                        "type": "TCP_DST",
                        "tcpPort": <integer>
                    },
                    {
                        "type": "UDP_DST",
                        "udpPort": <integer>
                    },
                    {
                        "type": "IN_PORT",
                        "port": "<integer>"
                    }
                ]
            }
        }
    ]
}


2. **Blocking or Dropping Rule:**

{
    "flows": [
        {
            "priority": <integer>,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "<switch_id>",
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "<string>" // Example: "0x800" for IPv4
                    },
                    {
                        "type": "IPV4_SRC",
                        "ip": "<ip_address/mask>"
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "<ip_address/mask>"
                    }
                ]
            }
        }
    ]
}

Field Descriptions
priority (Mandatory): Priority level (higher numbers indicate higher priority). For blocking or firewall rules, assign a priority greater than 300.
timeout (Mandatory): Timeout in seconds after which the flow is removed (Default: 0).
isPermanent (Mandatory): "true" (always in quotes, per user preference).
deviceId (Mandatory): Switch ID where the rule is installed.
ethType (Mandatory): Ethernet protocol type. Use "0x800" for IPv4, "0x86DD" for IPv6, "0x806" for ARP.
IPV4_DST (Optional): IPv4 address in CIDR notation (e.g., "10.0.0.1/32"). Include only if explicitly mentioned.
IPV4_SRC (Optional): Source IP address (include only if explicitly mentioned).
IP_PROTO (Optional): Transport layer protocol (6 for TCP, 17 for UDP, 1 for ICMP).
TCP_DST (Optional): TCP destination port (e.g., 80 for HTTP).
UDP_DST (Optional): UDP destination port (e.g., 161 for SNMP).
IN_PORT (Optional): Incoming interface port number (use in port-based forwarding).
QUEUE (Optional): Use "QUEUE" with "queueId" to specify a QoS queue (queue ID is an integer, 0 is default).
OUTPUT (Optional): "OUTPUT" with "port" specifies the output port.
VLAN_ID (Optional): Use "L2MODIFICATION" with "subtype": "VLAN_ID" and "vlanId" to set a VLAN tag.

Rules for Translation
Each "priority" must be unique.
Set priority high (e.g., 1000) for queue-related rules.
Do not include VLAN-related fields unless explicitly mentioned in the intent.
Do not include optional fields unless explicitly mentioned in the intent.
Ensure valid ONOS-compliant JSON syntax.
Verify JSON structure before responding.
Always respond in valid JSON format only, without comments, explanations, or additional text.
If the intent cannot be mapped, return an empty JSON object {}."""

In [ ]:
CONFLICT_PROMPT_ONOS = """You are tasked with determining if two ONOS flow configuration JSONs directly conflict with each other. **You MUST base your decision SOLELY on the JSON content provided.** Do **NOT** infer intent, use semantic reasoning, or make assumptions beyond the given JSON structure.

---

### **Rules for Conflict Detection (STRICT and LITERAL COMPARISON):**

A **direct conflict** exists **ONLY IF** all the following conditions are met:

#### **1. Matching Traffic Characteristics (Exact or Overlapping Match Criteria)**
Both flows must match **overlapping traffic characteristics** for all the specified fields, meaning they could apply to the **exact same packets**. The following fields are checked:

**EtherType (`ETH_TYPE`)**  
   - Must be present in **both** flows and have **identical** values (e.g., `"0x800"` for IPv4, `"0x86DD"` for IPv6).
   - If missing in either flow, there is **no match**.

**Source and Destination IP (`IPV4_SRC` and `IPV4_DST`)**  
   - If both flows **specify** a source or destination, they must overlap (e.g., `"10.0.0.1/32"` and `"10.0.0.0/24"` overlap).
   - If **either** flow **omits** source or destination, it applies to **all** sources or destinations respectively.

**Protocol (`IP_PROTO`)**  
   - Must be present in **both** flows and have **identical** values (e.g., `6` for TCP, `17` for UDP).
   - If missing in either flow, there is **no match**.

**Transport Layer Ports (`TCP_SRC`, `TCP_DST`, `UDP_SRC`, `UDP_DST`)**  
   - Must be **identical** in both flows if present.
   - If one flow includes a port filter and the other does not, there is **no match**.

**Incoming Port (`IN_PORT`)**  
   - If both flows specify `IN_PORT`, they must be identical.
   - If missing in either flow, there is **no match**.

**Wildcard Matching (Implicit Behavior for Missing Fields)**  
   - If a flow **specifies a field** (e.g., `IPV4_SRC`), it applies **only** to that source.
   - If a flow **does not specify a field** (e.g., `IPV4_SRC`), it applies to **all sources**.
   - General vs. specific distinctions are not classified, but missing fields imply generality (i.e., match all). Conflicts are determined based on whether the two match sets could overlap.
---

#### **2. Contradictory Actions**
Flows that **match the same traffic** only conflict if their actions contradict in one of the following ways:

**Different Output Ports (`OUTPUT` instruction)**  
   - If one flow **forwards** traffic to port `X` and another to port `Y`, it is a **conflict**.

**One Flow Drops, One Flow Forwards**  
   - If one flow **omits `"treatment"`** (implying a drop) and another **forwards traffic**, it is a **conflict**.

**Different Queue Assignments (`QUEUE` instruction)**  
   - If the flows **assign different `queueId` values**, it is a **conflict**.

**Note on Additional Match Fields:**  
   - If one flow includes additional match fields, a conflict may still exist if the effective match space overlaps. Extra fields do not automatically prevent a conflict.

---

#### **3. Priority is Irrelevant for Conflict Detection**
- `priority` **does not** determine a conflict.
- Even if one rule **overrides** another, they **do not conflict** unless their actions contradict.

---

### **Input Format**
You will be provided with **two ONOS JSON flow rules** in the following format:

**Flow 1:**
```json
<JSON for Flow 1>

**Flow 2:**
<JSON for Flow 2>

### **Expected Output Format**
Respond strictly in valid JSON format, using the schema below:

{
    "conflict_status": <integer>,
    "conflict_explanation": "<conflict explanation, if any>"
}

Field Descriptions:
   - conflict_status should be 1 if a direct conflict exists, 0 otherwise.
   - conflict_explanation → A brief explanation if a conflict exists, otherwise an empty string "".

NO EXTRA TEXT, COMMENTS, OR EXPLANATIONS OUTSIDE JSON.
DO NOT return 1 unless you are certain of a direct conflict.
Follow strict field-by-field comparison rules—NO inference beyond given data."""

In [ ]:
SLICING_PROMPT = """You are tasked with analyzing a natural language intent to determine if it contains a command to create or use a queue/slice in a openflow switch. You should respond in JSON format. 

### Rules for Interpretation:
1. **Queue/Slice Detection:**  
   - The intent is considered related to queue/slice if it contains commands such as:
     - "create queue", "create slices", "slice the network", "implement slicing", "slice the flow", "make flowspace slicing", "do slicing", "slice", "implement queue", "do queuing", "assign queue", "assign slice", or any similar phrasing.
   - If the intent does not mention creating or using a queue/slice, set the field `"use_queue"` to `0`. 

2. **Switch and Port Identification:**  
   - If the intent specifies a **switch ID** (e.g., "switch 4" or "openflow:4" or "node 4" or "openflow 4"), populate the `"switch_id"` field with its value.  
   - If the intent specifies a **Queue ID or slice ID** (e.g., "queue 4" or "4th queue" or "fourth queue" or "slice 1" or "first slice"), populate the `"queue_id"` field with its value.  
   - If the intent specifies a **port ID** (e.g., "port 2" or "interface 2" or "ethernet 2" or "output node connector 2" or "second port" or "second interface"), populate the `"port_id"` field with its value. If there are multiple instances of "port_id" present, take the one which indicates output port or outgoing interface.
   - If the intent does not specify a switch ID or queue ID or port ID, set the respective field to an empty string (`""`).

3. **Output Format:**  
   - Respond strictly in valid JSON format adhering to the following schema:

```json
{
  "use_queue": <integer>,
  "switch_id": "<string>",
  "queue_id": "<string>",
  "port_id": "<string>"
}

Field Description:
-use_queue: 1 if the intent commands to create or use a queue/slice, 0 otherwise.
-switch_id: Switch ID if specified in the intent, otherwise "".
-queue_id: Queue ID if specified in the intent, otherwise "".
-port_id: Port ID if specified in the intent, otherwise "". If there are mutiple instances of "port_id" present, take the one which indicates the output port or outgoing interface.

4. No Additional Text:
Do not include any comments, explanations, or outputs outside the JSON format.

Example Inputs and Outputs:
    1. Input Intent:
    "Create a queue in switch 4 on port 3 for slicing the flow."
    Output:
    {
    "use_queue": 1,
    "switch_id": "switch 4",
    "queue_id": "",
    "port_id": "port 3"
    }

    2. Input Intent:
    "Send all video traffic through queue 0 of openflow:2."
    Output:
    {
    "use_queue": 1,
    "switch_id": "openflow 2",
    "queue_id": 0,
    "port_id": ""
    }

    3. Input Intent:
    "Configure switch 5 for traffic management."
    Output:
    {
    "use_queue": 0,
    "switch_id": "switch 5",
    "queue_id": "",
    "port_id": ""
    }

    4. Input Intent:
    "Monitor traffic flow on port 1."
    Output:
    {
    "use_queue": 0,
    "switch_id": "",
    "queue_id": "",
    "port_id": "port 1"
    }

   5. Input Intent:
    "In switch 3, if the incoming traffic in port 1 is TCP traffic destined for port 80, then pass it via interface 2, assigning it to queue 0 for prioritized handling."
    Output:
    {
    "use_queue": 1,
    "switch_id": switch 3,
    "queue_id": 0,
    "port_id": "interface 2"
    }

Respond with the appropriate JSON strictly following these rules and format."""

In [7]:
my_models_translate_real = [
"codestral",
"command-r",
"huihui_ai/qwq-abliterated"
]

my_models_conflict_real = [
"qwq",
"huihui_ai/qwq-fusion"
]

context_examples = [3, 6]

default_model = "llama2"

ollama_embedding_url = "http://10.23.7.63:11434"
ollama_server_url = "http://10.23.7.63:11435"  

ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

In [8]:
# Load custom dataset from CSV
custom_dataset = pd.read_csv('ONOS_intent_translation_dataset_for_LLM_Evaluation.csv')

# Ensure proper column names and format
if not {'instruction', 'output'}.issubset(custom_dataset.columns):
    raise ValueError("The dataset must have 'instruction' and 'output' columns.")

# Split into train and test (50/50 split for example)
#trainset, testset = train_test_split(custom_dataset, test_size=0.5, random_state=42, shuffle=True)
trainset = custom_dataset

In [9]:
def run_LLM_conflict(existing_intent_flow_json, new_intent_flow_json):
    system_prompt = CONFLICT_PROMPT_ONOS

    for model in my_models_conflict_real:
        count = 0
        while True:
            count+=1
            try:
                time.sleep(0.1)

                response = client.generate(model=model,
                    options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                    stream=False,
                    system=system_prompt,
                    prompt=f"Flow 1:\n{json.dumps(existing_intent_flow_json, indent=2)}\n\nFlow 2:\n{json.dumps(new_intent_flow_json, indent=2)}",
                    format='json'
                )

                output = response['response'].strip()

                response_json = json.loads(output)

                if 'conflict_status' not in response_json:
                    #print("\nWarning: 'conflict_status' key is missing in the response.\n")
                    break
                else:
                    valid_conflict_response = True
                    conflict_status = response_json.get('conflict_status', 0)
                    # Ensure conflict_status is an integer
                    if isinstance(conflict_status, str):
                        conflict_status = int(conflict_status)

                    return valid_conflict_response, conflict_status, response_json['conflict_explanation']             

            except Exception as e:
                print("Exception found: ", e)
                sys.stdout.flush()
                if(count<15):
                    continue
                else:
                    print("\n",model, " failed to produce valid JSON for conflict info after 15 tries. Going to next model\n")
                    break               
    
    return False, None, None

In [10]:
def conflict(device_id, new_intent_flow_json):
    
    existing_flows = get_flows_for_device_ONOS(device_id)
    #existing_flows = manual_flows_by_device.get(device_id, {}).get("flows", [])

    for existing_flow in existing_flows:
        valid_conflict_response, conflict_status, conflict_details = run_LLM_conflict(existing_flow, new_intent_flow_json)

        if (valid_conflict_response == False):
            return 2, None, None   #2 means here that LLM could not generate valid JSON for conflict reporting
        elif (conflict_status == 1):
            return conflict_status, conflict_details, existing_flow
        
    return 0, None, None

In [ ]:
def run_LLM_IBN(intent, device_id):

    for num_examples in context_examples:
        for model in my_models_translate_real:
            
            example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
                [{"instruction": trainset.iloc[0]["instruction"], "output": trainset.iloc[0]["output"]}],
                ollama_emb,
                Chroma,
                input_keys=["instruction"],
                k=num_examples,
                vectorstore_kwargs={"fetch_k": min(num_examples, len(trainset))}
                )
            # Clear and add all remaining examples from the trainset
            example_selector.vectorstore.reset_collection()
            
            for _, row in trainset.iterrows():
                example_selector.add_example({
                    "instruction": row["instruction"],
                    "output": row["output"]
                })
            
            system_prompt = TRANSLATION_PROMPT_ONOS
            count = 0

            while True:
                count+=1
                try:
                    time.sleep(0.1)
                    if num_examples > 0:
                        examples = example_selector.select_examples({"instruction": intent})
                        example_str = "\n\n\n".join(map(lambda x: "Input: " + x["instruction"] + "\n\nOutput: " + x["output"], examples))
                        system_prompt += example_str + "\n\n\n"  
                    
                    response = client.generate(model=model,
                        options={'temperature': 0.6, 'num_ctx': 8192, 'top_p': 0.3, 'num_predict': 1024, 'num_gpu': 99},
                        #options={'device': 'cpu'},
                        stream=False,
                        system=system_prompt,
                        prompt=intent,
                        format='json'
                    )
                    actual_output = response['response']
                    #print("\nTranslated by: ", model)
                    break
                
                except Exception as e:
                    print("Exception on Input: ", e)
                    print("\nCheck example_str same or not: \n",example_str)
                    sys.stdout.flush()
                    if(count<15):
                        continue
                    else:
                        print("\n",model, " failed to produce valid JSON for translation info after 15 tries. Going to next model\n")
                        break 
            try:
                
                flow_json = json.loads(actual_output)
                
                for flow in flow_json.get("flows", []):  # Iterate over all flows
                    flow["deviceId"] = device_id  # Replace the device ID

                #print(json.dumps(flow_json))

                return flow_json
            
            except Exception as e:
                print("Exception found: ", e)

In [ ]:
def run_LLM_Slice(intent):

    system_prompt = SLICING_PROMPT
    
    for model in my_models_translate_real:     
        try:
            time.sleep(0.1)             
            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                #options={'device': 'cpu'},
                stream=False,
                system=system_prompt,
                prompt=intent,
                format='json'
            )
            
            output = response['response'].strip()
            response_json = json.loads(output)
            
            print("\nCheckpoint*******Exiting LLM Slicing detection\n\n******")
            return response_json            
        
        except Exception as e:
            print("Exception found: ", e)
            sys.stdout.flush()
            continue

In [ ]:
def end_to_end_IBN(intent):

    current_time = time.time()
    device_id = extract_switch_id_ONOS(intent)
    intent_JSON = run_LLM_IBN(intent, device_id)
    conflict_status, conflict_details, which_flow_conflict = conflict(device_id, intent_JSON)

    if(conflict_status == 2):
        print("\nCheck Conflict Detection Module, LLM did not produce a valid JSON for conflict detection.\n")

    elif (conflict_status == 1):
        proc_time_s = (time.time() - current_time)
        print(f"The intent conflicts with the existing flow rule:\n")
        print(which_flow_conflict, "\n\nExplanation: ", conflict_details)
        print("\nTime taken: ", proc_time_s)

    else:
        flow_id = push_flow_rule(device_id, intent_JSON)
        verification_status = verify_flow_rule(device_id, flow_id)
        if(verification_status == True):
            proc_time_s = (time.time() - current_time)
            print("\n\nSuccessfully translated, conflict checked and installed the rule in ONOS SDN Controller. Time taken: ", proc_time_s)
    
    return conflict_status, intent_JSON